## Starting from basics

In [ ]:
from langchain.llms import OpenAI

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
llm = OpenAI(openai_api_key = os.getenv("OPEN_API_KEY"), temperature = 0.6) #By default calls chatgpt 3.5
"""# temperature lies b/w 0 and 1. the more towards one, the answers generated by it differs more. So, more creative.
# More the value of temparature more risk (may generate wrong output)"""

In [ ]:
text = "What is the capital of Andhra Pradesh?"

print(llm.predict(text))

In [ ]:
from langchain import HuggingFaceHub
os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.getenv("HUG_API_KEY")
llm_huggingface = HuggingFaceHub(repo_id = "google/flan-t5-base", model_kwargs = {"temperature":0, "max_length":64})

In [ ]:
output = llm_huggingface.predict(text)
print(output)

Here, if we see it is just a simple output. Not a full sentence.

In [ ]:
print(llm_huggingface.predict("Plase write a poem on my girlfriend"))

In [ ]:
print(llm.predict("Plase write a poem on my girlfriend"))

## Prompt templates and LLM Chains

In [ ]:
from langchain.prompts import PromptTemplate

In [ ]:
prompt_template = PromptTemplate(input_variables=['country'], template='Tell me the capital of this {country}')

prompt_template.format(country = 'India')

In [ ]:
from langchain.chains import LLMChain #Combine the inputs and models - create a chain

chain = LLMChain(llm=llm, prompt = prompt_template)
print(chain.run("India"))

## Combining Multiple chains using simple Sequential chain

In [ ]:
capital_prompt = PromptTemplate(input_variables=["country"],
                                template = "Please tell me the capital of the {country}")
capital_chain = LLMChain(llm=llm, prompt = capital_prompt)

famous_prompt = PromptTemplate(input_variables=['capital'],
                               template="Suggest me some places to visit in {capital}")
famous_chain = LLMChain(llm=llm, prompt=famous_prompt)

In [ ]:
from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(chains=[capital_chain, famous_chain])

In [ ]:
print(chain.run("Australia"))

In [ ]:
# Sequential Chain
capital_prompt = PromptTemplate(input_variables=["country"],
                                template = "Please tell me the capital of the {country}")
capital_chain = LLMChain(llm=llm, prompt = capital_prompt, output_key = "capital")

famous_prompt = PromptTemplate(input_variables=['capital'],
                               template="Suggest me some places to visit in {capital}")
famous_chain = LLMChain(llm=llm, prompt=famous_prompt, output_key="places")

In [ ]:
from langchain.chains import SequentialChain
chain = SequentialChain(chains=[capital_chain, famous_chain], 
                        input_variables=['country'], 
                        output_variables=['capital', 'places'])

In [ ]:
chain({'country':'India'})

## Chatmodels with ChatOpenAI

In [ ]:
from langchain.chat_models import ChatOpenAI

In [ ]:
from langchain.schema import HumanMessage, SystemMessage, AIMessage

In [ ]:
Chatllm = ChatOpenAI(openai_api_key = os.getenv("OPEN_API_KEY"), temperature = 0.6, model = 'gpt-3.5-turbo') #By default calls chatgpt 3.5

In [ ]:
Chatllm

In [ ]:
#System message - How do you want system to act like?
#HumanMessage - Actual input
Chatllm([
    SystemMessage(content="You are a comedian AI assistant"),
    HumanMessage(content="Please provide some comedy punchlines on AI")
])

## Prompt Template + LLM + Output Parsers

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [ ]:
class Commaseparatedoutput(BaseOutputParser):
    def parse(self, text:str):
        return text.strip().split(",")

In [ ]:
#System template
template = "You are a helpful assistant. When the user gives any input, you should generate 5 words synonyms in a comma separated list"
human_template = "{text}"
chatprompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template)
])

In [ ]:
#Chaining up
chain = chatprompt|Chatllm|Commaseparatedoutput()

In [ ]:
chain.invoke({"text":"intelligent"})